In [ ]:
import asyncio
from datetime import datetime, timedelta
from alpaca.data.stream import StockDataStream
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

API_KEY = 'your_api_key'
SECRET_KEY = 'your_secret_key'
BASE_URL = 'https://paper-api.alpaca.markets'

trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)
data_stream = StockDataStream(API_KEY, SECRET_KEY)
historical_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)


In [ ]:

class CombinedTrader:
    def __init__(self, trading_client, data_stream, historical_client, symbols, balance):
        self.trading_client = trading_client
        self.data_stream = data_stream
        self.historical_client = historical_client
        self.symbols = symbols
        self.balance = balance
        self.symbol_data = {symbol: [] for symbol in symbols}
        self.symbol_allocation = self.allocate_balance()
        self.initial_data_filled = {symbol: False for symbol in symbols}
        self.market_open_time = datetime.utcnow().replace(hour=13, minute=30, second=0, microsecond=0)  # 9:30 AM ET
        self.pre_market_start_time = self.market_open_time - timedelta(minutes=15)

    def allocate_balance(self):
        allocation = self.balance / len(self.symbols)
        return {symbol: allocation for symbol in self.symbols}

    def get_historical_bars(self, symbol):
        start_time = self.pre_market_start_time
        end_time = datetime.utcnow()
        request_params = StockBarsRequest(
            symbol_or_symbols=symbol,
            timeframe=TimeFrame.Minute,
            start=start_time,
            end=end_time
        )
        bars = self.historical_client.get_stock_bars(request_params).df
        return bars

    async def on_bar(self, bar):
        symbol = bar.symbol
        current_price = bar.close
        self.symbol_data[symbol].append(bar)

        if not self.initial_data_filled[symbol] and len(self.symbol_data[symbol]) >= 15:
            self.initial_data_filled[symbol] = True
            # Trigger your strategy initialization here
            print(f"Initial data filled for {symbol}. Starting strategy execution.")
            await self.run_strategy(symbol)

        if self.initial_data_filled[symbol]:
            # Continue running your strategy with new data
            await self.run_strategy(symbol)

    async def run_strategy(self, symbol):
        # Example of running your strategy with the latest data
        print(f"Running strategy for {symbol} with latest data.")

    async def place_market_order(self, symbol, qty, side):
        order_data = MarketOrderRequest(
            symbol=symbol,
            qty=qty,
            side=side,
            time_in_force=TimeInForce.DAY
        )
        self.trading_client.submit_order(order_data)
        print(f"Placed {side} order for {qty} shares of {symbol}")

    async def run(self):
        # Fetch historical data to fill initial window
        for symbol in self.symbols:
            historical_data = self.get_historical_bars(symbol)
            self.symbol_data[symbol].extend(historical_data.to_dict('records'))
            if len(self.symbol_data[symbol]) >= 15:
                self.initial_data_filled[symbol] = True
                # Trigger your strategy initialization here
                print(f"Initial data filled for {symbol} with historical data.")
                await self.run_strategy(symbol)

        # Subscribe to real-time data stream
        for symbol in self.symbols:
            self.data_stream.subscribe_bars(self.on_bar, symbol)

        await self.data_stream.run()


In [ ]:

symbols = ['AAPL', 'TSLA']
trader = CombinedTrader(trading_client, data_stream, historical_client, symbols, 10000)
asyncio.run(trader.run())
